In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [3]:
csv_file = "Resources/twtr_clean_all.csv"
twtr_all = pd.read_csv(csv_file)
twtr_all.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701670,0.0,0.0
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,27925307,0.0,0.0
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113941,0.0,0.0
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316755,0.0,0.0
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688325,0.0,0.0


In [4]:
twtr_all_df = twtr_all[['Date', 'Open', 'High','Low','Close','Volume']].copy()
twtr_all_df.head()

,Date,Open,High,Low,Close,Volume
0,2013-11-07,45.099998,50.090000,44.000000,44.900002,117701670
1,2013-11-08,45.930000,46.939999,40.685001,41.650002,27925307
2,2013-11-11,40.500000,43.000000,39.400002,42.900002,16113941
3,2013-11-12,43.660000,43.779999,41.830002,41.900002,6316755
4,2013-11-13,41.029999,42.869999,40.759998,42.599998,8688325


In [5]:
json_file = "Resources/twtr_dates.json"
twtr_dates_df = pd.read_json(json_file)
twtr_dates_df.head()

,Date,Headline
0,2022-01-31,January 31: Musk begins building up his Tw...
1,2022-03-14,March 14: Musk’s Twitter stake tops 5%
2,2022-03-24,March 24: Asking whether Twitter should ch...
5,2022-04-04,April 4: Surprise! Musk becomes Twitter’s ...
6,2022-04-05,April 5: Musk agrees to join the board


In [6]:
import psycopg2

In [7]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'twitter_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [8]:
engine.table_names()

['twtr_all', 'twtr_dates']

In [15]:
# twtr_all_df.to_sql(name='twtr_all', con=engine, if_exists='append', index=False)

In [16]:
# twtr_dates_df.to_sql(name='twtr_dates', con=engine, if_exists='append', index=False)

In [11]:
pd.read_sql_query('select * from twtr_all', con=engine).head()

,Date,Open,High,Low,Close,Volume
0,2013-11-07,45.0,50.0,44.0,45.0,117701670
1,2013-11-08,46.0,47.0,41.0,42.0,27925307
2,2013-11-11,41.0,43.0,39.0,43.0,16113941
3,2013-11-12,44.0,44.0,42.0,42.0,6316755
4,2013-11-13,41.0,43.0,41.0,43.0,8688325


In [12]:
pd.read_sql_query('select * from twtr_dates', con=engine).head()

,Date,Headline
0,2022-01-31,January 31: Musk begins building up his Tw...
1,2022-03-14,March 14: Musk’s Twitter stake tops 5%
2,2022-03-24,March 24: Asking whether Twitter should ch...
3,2022-04-04,April 4: Surprise! Musk becomes Twitter’s ...
4,2022-04-05,April 5: Musk agrees to join the board


In [13]:
sql_join = r"""select twtr_dates."Headline",
	twtr_all."Open",
	twtr_all."High",
	twtr_all."Low",
	twtr_all."Close"
from "twtr_dates"
join "twtr_all"
on twtr_all."Date" = twtr_dates."Date" """

In [14]:
pd.read_sql_query(sql_join, con=engine)

,Headline,Open,High,Low,Close
0,January 31: Musk begins building up his Tw...,35.0,38.0,35.0,38.0
1,March 14: Musk’s Twitter stake tops 5%,33.0,34.0,32.0,33.0
2,March 24: Asking whether Twitter should ch...,38.0,39.0,38.0,39.0
3,April 4: Surprise! Musk becomes Twitter’s ...,48.0,51.0,47.0,50.0
4,April 5: Musk agrees to join the board,54.0,55.0,51.0,51.0
5,April 14: Musk offers to buy Twitter and ‘...,48.0,49.0,45.0,45.0
6,April 21: Musk lines up $46.5 billion in f...,46.0,48.0,46.0,47.0
7,April 25: Twitter agrees to sell itself to...,51.0,52.0,50.0,52.0
8,April 29: Musk cashes out billions in Tesl...,49.0,50.0,49.0,49.0
9,May 4: With a little help from his billion...,49.0,49.0,48.0,49.0


In [ ]:
# For pgAdmin

# select * from twtr_dates;

# select twtr_dates.Date, twtr_all.Volume
# from twtr_dates
# inner join twtr_all
# on twtr_all.Date = twtr_dates.Date;

# select "Headline"
# from "twtr_dates"
# where "Date" = '2022-04-05';

# select twtr_dates."Date",
# 	twtr_dates."Headline",
# 	twtr_all."Open",
# 	twtr_all."High",
# 	twtr_all."Low",
# 	twtr_all."Close"
# from "twtr_dates"
# join "twtr_all"
# on twtr_all."Date" = twtr_dates."Date";

# select twtr_dates."Headline",
# 	twtr_all."Open",
# 	twtr_all."High",
# 	twtr_all."Low",
# 	twtr_all."Close"
# from "twtr_dates"
# join "twtr_all"
# on twtr_all."Date" = twtr_dates."Date";